In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe
import datetime

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('crowds_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'desc_location']]

### Read the current data from SQL

In [43]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_crowds'

In [44]:
crowds_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['crowds_id'])
              .fillna(value = np.nan, axis = 1))

In [45]:
format_datetime(crowds_sql, 'encounter_timestamp')

In [46]:
float_cols = ['patroncount', 'in_playground']
for c in float_cols:
    crowds_sql[c] = crowds_sql[c].astype(float)

In [47]:
crowds_sql.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough
0,06-01-2020 10:54:14,Q-12,12.0,1.0,Approached the crowd; they ignored the employee,Playground,NaN,Q005-11,Queens
1,06-01-2020 10:22:51,X-14,11.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,Playing soccer,X147A-ZN01,Bronx
2,05-31-2020 16:31:20,Q-13,13.0,0.0,Approached the crowd; they ignored the employee,Skate Park,NaN,Q108,Queens
3,05-31-2020 15:04:12,Q-07,30.0,1.0,Approached the crowd; they complied with instr...,Basketball court,NaN,Q397,Queens
4,05-31-2020 14:38:52,X-02,15.0,0.0,Did not approach the crowd; the crowd remains,Baseball field,Central communication was called,X255,Bronx


In [48]:
sql_cols = list(crowds_sql.columns.values)

In [49]:
hash_rows(crowds_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [20]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [21]:
sheet = client.open('Crowds_Combined')

In [22]:
ws = sheet.worksheet('Sheet1')

In [23]:
crowds = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [24]:
#Remove the rows where there timestamp is null because these sheets have extra rows full of nulls
crowds = crowds[crowds['encounter_timestamp'].notnull()]

In [25]:
format_datetime(crowds, 'encounter_timestamp')

In [26]:
crowds.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,desc_location,borough
1,06-02-2020 18:46:01,Q-01,100.0,No,Did not approach the crowd; the crowd remains,Soccer field,NaN,Q004 | Astoria Park,Queens
2,06-02-2020 17:47:38,Q-13,20.0,No,Did not approach the crowd; the crowd remains,Open field/multi-purpose play area,20 people playing soccer in idlewild cricket f...,Q392-ZN01 | Idlewild Park Zone 1,Queens
3,06-02-2020 17:32:52,Q-06,20.0,No,Did not approach the crowd; the crowd remains,Chess area,Chess players,Q207 | MacDonald Park,Queens
4,06-02-2020 17:10:48,X-14,30.0,No,Did not approach the crowd; the crowd remains,Soccer field,Soccer game,X147A-ZN01 | Starlight Park Zone 1,Bronx
5,06-02-2020 15:50:38,Q-14,7.0,No,Approached the crowd; they complied with instr...,Handall court,B105,Q162 | Rockaway Beach and Boardwalk,Queens


In [27]:
yesno = ['in_playground']

In [28]:
yesno_cols(crowds, yesno)

In [29]:
crowds = crowds.merge(site_ref, how = 'left', on = 'desc_location')[sql_cols]

In [30]:
hash_rows(crowds, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

In [31]:
crowds.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough,row_hash
0,06-02-2020 18:46:01,Q-01,100.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,NaN,Q004,Queens,806514fedb87fd78073c04cf44ac24544ca3baa3a3244f...
1,06-02-2020 17:47:38,Q-13,20.0,0.0,Did not approach the crowd; the crowd remains,Open field/multi-purpose play area,20 people playing soccer in idlewild cricket f...,Q392-ZN01,Queens,03838f02eb4e5eb00112b8df63963dcba2bc089c778fa6...
2,06-02-2020 17:32:52,Q-06,20.0,0.0,Did not approach the crowd; the crowd remains,Chess area,Chess players,Q207,Queens,9e5e52ceae175cdb0677c6e1e6aed9e67d0f220dcceedf...
3,06-02-2020 17:10:48,X-14,30.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,Soccer game,X147A-ZN01,Bronx,79f649aea0225c65b53df75326f3e8985dacad29b72ca8...
4,06-02-2020 15:50:38,Q-14,7.0,0.0,Approached the crowd; they complied with instr...,Handall court,B105,Q162,Queens,ebc0a91d8c613c59ed22b3432dc24078a98dced5c6eaf7...


### Find the deltas based on the row hashes

In [50]:
crowds_deltas = (check_deltas(new_df = crowds, old_df = crowds_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [51]:
crowds_deltas.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough,dml_verb
0,06-02-2020 18:46:01,Q-01,100.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,NaN,Q004,Queens,None
1,06-02-2020 17:47:38,Q-13,20.0,0.0,Did not approach the crowd; the crowd remains,Open field/multi-purpose play area,20 people playing soccer in idlewild cricket f...,Q392-ZN01,Queens,None
2,06-02-2020 17:32:52,Q-06,20.0,0.0,Did not approach the crowd; the crowd remains,Chess area,Chess players,Q207,Queens,None
3,06-02-2020 17:10:48,X-14,30.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,Soccer game,X147A-ZN01,Bronx,None
4,06-02-2020 15:50:38,Q-14,7.0,0.0,Approached the crowd; they complied with instr...,Handall court,B105,Q162,Queens,None


In [52]:
crowds_inserts = crowds_deltas[crowds_deltas['dml_verb'] == 'I'][sql_cols]

In [53]:
crowds_inserts.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough


In [54]:
crowds_inserts.to_sql('tbl_dpr_crowds', engine, index = False, if_exists = 'append')

In [55]:
crowds_updates = crowds_deltas[crowds_deltas['dml_verb'] == 'U'][sql_cols]

In [56]:
crowds_updates.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough


In [58]:
sql_update(crowds_updates, 'tbl_dpr_crowds', engine, ['encounter_timestamp', 'site_id'])